### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed. 



In [2]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>7.2.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [4]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

import com.aerospike.client.cdt.ListPolicy;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.ListOrder;
import com.aerospike.client.cdt.ListWriteFlags;
import com.aerospike.client.cdt.ListReturnType;

import java.util.HashMap;
import java.util.Map;

import java.util.Random;

AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);



In [5]:
Key myKey = new Key("test", "testset", "mykey");
String binName = "myList";
ListPolicy lPolicy = new ListPolicy(ListOrder.UNORDERED, ListWriteFlags.DEFAULT);
client.delete(null,myKey);
client.operate(null, myKey, ListOperation.append(lPolicy, binName, Value.get(10))); 
client.operate(null, myKey, ListOperation.append(lPolicy, binName, Value.get(5))); 
client.operate(null, myKey, ListOperation.append(lPolicy, binName, Value.get(15))); 
client.operate(null, myKey, ListOperation.append(lPolicy, binName, Value.get(20))); 
System.out.println("My List: "+ client.get(null, myKey)); 
client.operate(null, myKey, ListOperation.pop(binName, 1) );  //Pop at index 1, (value = 5)
System.out.println("My List after pop: "+ client.get(null, myKey)); 

My List: (gen:4),(exp:450852284),(bins:(myList:[10, 5, 15, 20]))
My List after pop: (gen:5),(exp:450852285),(bins:(myList:[10, 15, 20]))


# Cleanup

In [6]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"